# 🖥️ Thực tiễn Hệ điều hành (Practical OS)

**Tác giả bài giảng gốc:** Nguyễn Tuấn Đức

Tài liệu này giúp bạn hiểu rõ các khái niệm cốt lõi về Hệ điều hành thông qua các **ví dụ đời thực** và **hình ảnh minh họa**. Hãy quên đi những định nghĩa khô khan, chúng ta sẽ học bằng cách "nhìn thấy" nó hoạt động!

---

## 📚 Bài 1: Hệ điều hành là gì?

### 🏬 Ví dụ minh họa: Siêu thị & Cảnh sát

Hãy tưởng tượng máy tính của bạn là một **thành phố**. Trong thành phố đó:

| Vai trò | Trong đời thực | Trong máy tính |
|:---|:---|:---|
| **Siêu thị** 🛒 | Cung cấp thực phẩm, đồ dùng cho mọi người | Hệ điều hành cung cấp các "dịch vụ" (đọc file, in ấn, kết nối mạng...) cho các ứng dụng |
| **Cảnh sát** 👮 | Giữ gìn trật tự, không cho ai chiếm hết đường | Hệ điều hành quản lý tài nguyên (CPU, RAM), không để ứng dụng nào "chiếm hết" |

**Vậy Hệ điều hành = Siêu thị + Cảnh sát!**

### 🏢 Thread và Process: Công ty và Nhân viên

| Khái niệm | Ví dụ đời thực | Giải thích |
|:---|:---|:---|
| **Process (Tiến trình)** | Một **công ty** | Có trụ sở riêng (bộ nhớ riêng), có nhân viên, có tài liệu riêng |
| **Thread (Luồng)** | Một **nhân viên** trong công ty | Làm việc trong công ty, dùng chung tài liệu với đồng nghiệp nhưng có bàn làm việc riêng |

**Ví dụ cụ thể:**
- Mở Chrome = Tạo một **Process** (công ty)
- Mỗi tab Chrome = Một **Thread** (nhân viên) trong công ty đó
- Các tab dùng chung bộ nhớ của Chrome nhưng xử lý nội dung riêng biệt

### ⏰ Lập lịch (Scheduling): Ai được dùng CPU?

Hãy tưởng tượng CPU như một **phòng họp VIP** chỉ có 1-4 ghế (số lõi CPU). Nhưng có hàng trăm người (thread) muốn vào họp!

**Scheduler** (Bộ lập lịch) đóng vai trò như **cô lễ tân**:
- Quyết định ai được vào phòng họp
- Đảm bảo công bằng: Không ai được ngồi cả ngày
- Ưu tiên người cần gấp (ứng dụng cần phản hồi nhanh như Notepad)

**Các trạng thái của một Thread:**
```
🟢 Running: Đang trong phòng họp (đang dùng CPU)
🟡 Ready: Đang xếp hàng chờ (sẵn sàng nhưng chưa tới lượt)
🔴 Waiting: Đang ngủ (chờ file đọc xong, chờ mạng...)
```

---
## ⚔️ Bài 2: Khi các Thread "đánh nhau" (Race Condition)

### 🏦 Ví dụ: Hai người cùng rút tiền từ một tài khoản

Giả sử tài khoản có **1.000.000đ**. Hai người (Thread A và Thread B) cùng rút **700.000đ** cùng lúc:

| Bước | Thread A | Thread B | Số dư thực tế |
|:---|:---|:---|:---|
| 1 | Kiểm tra số dư: 1.000.000đ ✅ | | 1.000.000đ |
| 2 | | Kiểm tra số dư: 1.000.000đ ✅ | 1.000.000đ |
| 3 | Rút 700.000đ | | 300.000đ |
| 4 | | Rút 700.000đ | **-400.000đ** 💥 |

**Kết quả:** Ngân hàng bị thâm hụt!

Đây gọi là **Race Condition** (Tình trạng đua) - kết quả phụ thuộc vào "ai chạy nhanh hơn".

### 🔒 Giải pháp: Critical Section (Miền găng)

**Miền găng** là đoạn code chỉ cho phép **1 người vào cùng lúc**.

Hãy tưởng tượng nó như **phòng thay đồ** ở hồ bơi:
- Chỉ 1 người được vào
- Người khác phải đứng chờ bên ngoài
- Khi xong việc mới mở cửa cho người tiếp theo

**3 điều kiện cho giải pháp tốt:**
1. **Mutual Exclusion (Loại trừ):** Chỉ 1 người trong phòng
2. **Progress (Tiến triển):** Nếu phòng trống, ai đến trước vào trước
3. **Bounded Waiting (Chờ có giới hạn):** Không ai phải chờ mãi mãi

---
## 🔧 Bài 3: Các công cụ đồng bộ hóa

### 🚪 Mutex: Khóa cửa nhà vệ sinh

```
pthread_mutex_lock(&khoa);   // 🔒 Vào và khóa cửa
    // Làm việc bên trong...
pthread_mutex_unlock(&khoa); // 🔓 Mở cửa đi ra
```

**Ai cầm khóa thì mới được vào, người khác phải đợi!**

### 📢 Event: Chuông báo thức

Dùng khi cần đảm bảo **thứ tự thực hiện**.

**Ví dụ: Nhà hàng**
- Đầu bếp (Producer): Nấu xong thì bấm chuông 📢
- Phục vụ (Consumer): Ngủ chờ, nghe chuông thì mang đồ ra

Nếu không có Event, phục vụ có thể mang đĩa trống ra bàn! 🍽️

### 🏨 Semaphore: Bãi đỗ xe

Giống Mutex nhưng cho phép **nhiều người** cùng vào (có giới hạn).

**Ví dụ:** Bãi đỗ xe có 5 chỗ
- Xe 1-5 vào: OK ✅
- Xe thứ 6: Phải đợi ngoài cổng ❌
- Khi xe 3 ra → Xe 6 được vào

```c
sem_init(&bai_do, 0, 5);  // Bãi có 5 chỗ
sem_wait(&bai_do);        // Xin vào (chỗ trống giảm 1)
// Đỗ xe...
sem_post(&bai_do);        // Ra khỏi (chỗ trống tăng 1)
```

---
## 💀 Bài 4: Deadlock - Khi ai cũng chờ ai

### 🍝 Ví dụ kinh điển: Bữa ăn của các triết gia

5 triết gia ngồi quanh bàn tròn, giữa mỗi 2 người có 1 chiếc đũa. Để ăn cần 2 đũa.

**Thảm họa xảy ra khi:**
- Triết gia 1: Cầm đũa bên trái, chờ đũa bên phải
- Triết gia 2: Cầm đũa bên trái, chờ đũa bên phải
- ... (tất cả đều cầm 1 đũa và chờ)

**Kết quả:** Tất cả chết đói! 💀

Đây là **Deadlock (Bế tắc)** - các luồng chờ nhau theo vòng tròn mãi mãi.

### 💡 Cách phòng tránh Deadlock

**1. Lock Ordering (Thứ tự khóa):**
Quy định: Ai cũng phải lấy đũa **số nhỏ trước**.
- Triết gia 1: Lấy đũa 1 → đũa 2 ✅
- Triết gia 5: Lấy đũa 1 (chờ) → đũa 5 ✅

**2. Read-Write Lock (Khóa Đọc-Ghi):**

| Tình huống | Cho phép? |
|:---|:---|
| Nhiều người cùng **đọc** sách | ✅ Được |
| Một người **viết** + nhiều người đọc | ❌ Không |
| Một người **viết** duy nhất | ✅ Được |

Vì đọc không làm hỏng dữ liệu nên cho phép đọc song song → Tăng hiệu năng!

---
## 🎯 Tóm tắt nhanh

| Khái niệm | Hình ảnh minh họa |
|:---|:---|
| Hệ điều hành | Siêu thị + Cảnh sát |
| Process | Công ty |
| Thread | Nhân viên |
| Scheduler | Cô lễ tân phòng họp |
| Race Condition | Hai người rút tiền cùng lúc |
| Mutex | Khóa cửa nhà vệ sinh |
| Semaphore | Bãi đỗ xe |
| Deadlock | Triết gia chờ đũa |

**Chúc bạn học tốt! 🚀**